### Scoring using the best model locally - Optional

The training environment in AML compute cluster can be different than the one locally. We need to create a new Conda environment in the repo so that model can be loaded locally. 

    conda env create -f conda_env.yml

Use the model file downloaded using automl_amlsdkv2_training.ipynb. 

If AML Compute Instance is used for local scoring, then the notebook kernel should be set to Python 3.8 - AzureML

If AML Compute Cluster is used for remote scoring, then the notebook kernel should be set to Python 3.10 - SDK v2

### Score using Compute Cluster in AML - Optional 

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

#### Provide values for your AML workspace

    subscription_id = "YOUR_SUBSCRIPTION_ID"
    resource_group = "YOUR_RESOURCE_GROUP"
    workspace = "YOUR_WORKSPACE_NAME"

In [ ]:

ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your Azure Machine Learning workspace
    subscription_id = "f1a8fafd-a8a3-46d8-bb5e-01deb63d275d"
    resource_group = "aml-rg"
    workspace = "aml-testws"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Set the model name created in automl_amlsdkv2_training.ipynb

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import ModelType

model_name = "wa_telco_customer_churn_model_best"
model_versions = ml_client.models.list(name=model_name)
latest_model = max(model_versions, key=lambda x: x.version)
latest_model.path

In [ ]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes
data_type = AssetTypes.MLTABLE
input_mode = InputOutputModes.RO_MOUNT
output_mode = InputOutputModes.RW_MOUNT

Get reference to test dataset from AML

In [ ]:
data_asset_name = "wa_telco_customer_churn_test_data_uri_file"
data_asset_version = "1.0"
data_asset = ml_client.data.get(name=data_asset_name, version=data_asset_version)

Create Input schema for scoring job

In [ ]:

#data_path = "./wa_telco_customer_churn_test_data_uri_file.csv"
inputs = {
    "input_data": Input(type=AssetTypes.URI_FILE, path=data_asset.path),
    "input_model": Input(type=AssetTypes.MLFLOW_MODEL, path=latest_model.path)
}

In [ ]:
inputs

Create Output schema for scoring job

In [ ]:
dataset_path = "wa-telco-customer-churn-predictions"
dataset_version = "1.0"
outputs = {
    "output_folder": Output(
        type=AssetTypes.URI_FOLDER,
        path=f"azureml://subscriptions/{ml_client.subscription_id}/resourcegroups/{ml_client.resource_group_name}/workspaces/{ml_client.workspace_name}/datastores/workspaceblobstore/paths/{dataset_path}",
        name = dataset_path,
        version = dataset_version
    )
}

Define scoring job using Command component

Use the same environment as the model was created in AutoML. 
    
        AzureML-AutoML:142

Please review score.py

In [ ]:
from azure.ai.ml import command, Input


# define the command
command_job = command(
    code="./",
    command="python score.py --input_model ${{inputs.input_model}} --input_data ${{inputs.input_data}} --output_folder ${{outputs.output_folder}}",
    experiment_name="wa-telco-customer-churn-prediction",
    inputs=inputs,
    outputs=outputs,    
    environment="AzureML-AutoML:142",
    compute="cpu-cluster",
)

# Submit the command
ml_client.jobs.create_or_update(command_job)

Please review the results of the Scoring job in Aml Workspace. 